In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date

In [61]:
data_player_gen = data_player(data_pool)
print(next(data_player_gen))

ticker       1332 JT            1333 JT          1334 JT             \
                last     volume    last volume      last     volume   
date                                                                  
2013-01-08  166.3266  1759800.0     NaN    NaN  144.9890   869000.0   
2013-01-09  165.4026   767800.0     NaN    NaN  144.9890   654000.0   
2013-01-10  167.2507  1503100.0     NaN    NaN  144.9890   661000.0   
2013-01-11  170.9468  2864200.0     NaN    NaN  147.8887  2351000.0   
2013-01-15  174.6430  2407400.0     NaN    NaN  152.7217  1919000.0   
2013-01-16  169.0987  3636700.0     NaN    NaN  154.6549  2856000.0   
2013-01-17  169.0987  3433500.0     NaN    NaN  152.7217  1726000.0   
2013-01-18  174.6430  1847500.0     NaN    NaN  155.6215  1135000.0   
2013-01-21  173.7189  1863300.0     NaN    NaN  153.6883   942000.0   
2013-01-22  170.9468  2203200.0     NaN    NaN  152.7217  2060000.0   
2013-01-23  164.4785  1396000.0     NaN    NaN  144.9890  2831000.0   
2013-0

In [5]:
def strategy(data ):
    target_position = [1]* len(SYMBOLS)
    return target_position

In [16]:
class backtest_engine():
    def __init__(self, data_pool, strategy):
        
        self.data_player_gen = self.data_player(data_pool)
        self.pnl = pd.DataFrame({'date':[], 'pnl':[]})
        self.cur_pos = None
        self.pos_his = pd.DataFrame(columns=['date'] + SYMBOLS)
        # self.pos_his.columns = 

        self.strategy = strategy

    def data_player(self, data_pool: pd.DataFrame):
        data_pool = data_pool[data_pool.index.to_series().between(DATE_START, DATE_END)]
        data_pool = data_pool.sort_index(axis=1, level=0)
        calendar = data_pool.index
        duration = WINDOW+1
        # print(data_pool)
        for i in range(duration-1, len(calendar)):
            data_out = data_pool.iloc[i-duration+1:i,:].loc[:, (SYMBOLS, slice(None))]
            yield data_out
    

    def portfolio_manager(self, target_position, dt, price):
        assert len(target_position)==len(SYMBOLS)
        print(price)
        prc = price[SYMBOLS].values
        open_pos = target_position*prc
        self.pnl = self.pnl.append({'date':dt, 'value':open_pos}, ignore_index=True)
        self.pos_his = self.pos_his.append(dict(zip(['date']+SYMBOLS, [dt]+target_position)), ignore_index=True)


    def end_backtest(self):
        self.pnl.to_csv('pnl.csv', index=False)
        self.pos_his.to_csv('position.csv', index=False)

    def trading_engine(self):
        for backtest_data in self.data_player_gen:
            # backtest_data = next(self.data_player_gen)
            tod_price = backtest_data.iloc[-1].loc[(SYMBOLS, 'last')]
            today_dt = backtest_data.iloc[-1].index
            his_data = backtest_data.iloc[:-1]
            print(backtest_data)
            target_position = self.strategy(his_data)
            self.portfolio_manager(target_position, today_dt, tod_price)

In [17]:
data_pool = pd.read_csv("data.csv", index_col='date', parse_dates=True).pivot(columns='ticker')
data_pool = data_pool.swaplevel(1,0, axis=1)

# backtest config
WINDOW = 14
SYMBOLS = data_pool.columns.get_level_values(0).unique()
DATE_START = datetime(2013,1,8)
DATE_END = datetime(2020,1,8)

INIT_CAPITAL = 1000000

assert WINDOW >0

bktst = backtest_engine(data_pool, strategy)



In [18]:

bktst.trading_engine()

ticker       1332 JT            1333 JT          1334 JT             \
                last     volume    last volume      last     volume   
date                                                                  
2013-01-08  166.3266  1759800.0     NaN    NaN  144.9890   869000.0   
2013-01-09  165.4026   767800.0     NaN    NaN  144.9890   654000.0   
2013-01-10  167.2507  1503100.0     NaN    NaN  144.9890   661000.0   
2013-01-11  170.9468  2864200.0     NaN    NaN  147.8887  2351000.0   
2013-01-15  174.6430  2407400.0     NaN    NaN  152.7217  1919000.0   
2013-01-16  169.0987  3636700.0     NaN    NaN  154.6549  2856000.0   
2013-01-17  169.0987  3433500.0     NaN    NaN  152.7217  1726000.0   
2013-01-18  174.6430  1847500.0     NaN    NaN  155.6215  1135000.0   
2013-01-21  173.7189  1863300.0     NaN    NaN  153.6883   942000.0   
2013-01-22  170.9468  2203200.0     NaN    NaN  152.7217  2060000.0   
2013-01-23  164.4785  1396000.0     NaN    NaN  144.9890  2831000.0   
2013-0

KeyboardInterrupt: 